In [1]:
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint


# Import API key
from api_keys import walkscore_key
url="https://api.walkscore.com/score?format=json&address="

In [2]:
tx_data_df = pd.read_csv("../resources/obesity_data_tx.csv")

tx_data_df.head()

,State,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Lat,Lon
0,TX,Houston,4835000-48201451401,4835000,48201451401,25.5,29.750367,-95.612367
1,TX,Garland,4829000-48113018138,4829000,48113018138,30.6,32.850897,-96.582081
2,TX,Mesquite,4847892-48113012500,4847892,48113012500,33.4,32.831949,-96.657915
3,TX,Dallas,4819000-48113004800,4819000,48113004800,38.5,32.743677,-96.817550
4,TX,Fort Worth,4827000-48439105800,4827000,48439105800,37.5,32.665385,-97.342787


In [4]:
tx_data_df["Walk Score"]=''

tx_data_df_cleaned=tx_data_df.dropna()
tx_data_df_cleaned.head()


,State,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Lat,Lon,Walk Score
0,TX,Houston,4835000-48201451401,4835000,48201451401,25.5,29.750367,-95.612367,
1,TX,Garland,4829000-48113018138,4829000,48113018138,30.6,32.850897,-96.582081,
2,TX,Mesquite,4847892-48113012500,4847892,48113012500,33.4,32.831949,-96.657915,
3,TX,Dallas,4819000-48113004800,4819000,48113004800,38.5,32.743677,-96.817550,
4,TX,Fort Worth,4827000-48439105800,4827000,48439105800,37.5,32.665385,-97.342787,


In [5]:
def getWalkScore(lat, lon, city): 
    query_url = url+city+"&lat="+lat+"&lon="+lon+"&wsapikey="+walkscore_key
    walk_response=requests.get(query_url)
    walk_json = walk_response.json() 
    return walk_json
 
for index, row in tx_data_df_cleaned.iterrows():
    try:

        lat=str(row["Lat"])
        long=str(row["Lon"])
        city=row["CityName"]
        walk_json=getWalkScore(lat,long,city)
        tx_data_df_cleaned.loc[index, "Walk Score"] = walk_json["walkscore"]
    except:
        print("Data not found. Skipping")
        pass
    


Data not found. Skipping
Data not found. Skipping
Data not found. Skipping


In [7]:
tx_data_df_cleaned


,State,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Lat,Lon,Walk Score
0,TX,Houston,4835000-48201451401,4835000,48201451401,25.5,29.750367,-95.612367,51
1,TX,Garland,4829000-48113018138,4829000,48113018138,30.6,32.850897,-96.582081,26
2,TX,Mesquite,4847892-48113012500,4847892,48113012500,33.4,32.831949,-96.657915,46
3,TX,Dallas,4819000-48113004800,4819000,48113004800,38.5,32.743677,-96.817550,85
4,TX,Fort Worth,4827000-48439105800,4827000,48439105800,37.5,32.665385,-97.342787,54
...,...,...,...,...,...,...,...,...,...
3108,TX,Houston,4835000-48201340100,4835000,48201340100,30.1,29.593659,-95.181274,36
3109,TX,San Antonio,4865000-48029140300,4865000,48029140300,39.6,29.386688,-98.483144,55
3112,TX,Corpus Christi,4817000-48355002706,4817000,48355002706,20.8,27.712757,-97.325859,36
3113,TX,Fort Worth,4827000-48439106600,4827000,48439106600,34.2,32.802436,-97.427495,32


In [8]:
tx_data_df_cleaned.to_csv("../resources/cities_complete.csv")
